In [ ]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point

dic = {
    'TSSs_l':'./data/gene/low_expressed.bed',
    'location':'./data/gene/GRCh37.gene.bed',
    'bam':'./data/051.bam',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'Lung':'./excel/Lung.bed',
    'Tcell':'./excel/Tcell.bed',
    'Liver':'./excel/Liver.bed'
    }

chr_to_id = {
    'chr1':'1','chr2':'2','chr3':'3','chr4':'4','chr5':'5','chr6':'6','chr7':'7','chr8':'8','chr9':'9','chr10':'10','chr11':'11','chr12':'12','chr13':'13','chr14':'14','chr15':'15','chr16':'16','chr17':'17','chr18':'18','chr19':'19','chr20':'20','chr21':'21','chr22':'22','chrX':'X','chrY':'Y'
}
bamfile = ps.AlignmentFile(dic['bam'],'rb')

up = 4000
down = 4000

length_cutoff = 140

count = 0

TSS_tissue = []
with open(dic['Tcell'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '1':
            TSS_tissue.append(TSS(ll[0],int(ll[1])))

for k, tss in enumerate(TSS_tissue):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    long_array = np.zeros(up+down, dtype=float)
    short_array = np.zeros(up+down, dtype=float)    
    plist = []
    for r in bamfile.fetch(chrom, start-500, end+500):
         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            ss = max(0, r.reference_start - start)
            ee = min(r.reference_start - start + abs(r.isize), end - start)
            if r.isize <= 150:
                for i in range(ss, ee):
                    short_array[i] += 1
            if r.isize > 150:
                for i in range(ss, ee):
                    long_array[i] += 1
    plt.plot([x for x in range(up+down)],[long_array[i]+short_array[i] for i in range(up+down)])
    plt.show()
    if k > 30:
        break
